Auteurs : Hachemin Pierre-Yves et Seys Thibaut

Date : 25/04/2018

Cours : IS3024AB - Data Mining

# Etude sur le transport en Ile-de-France

* [Introduction](#Introduction)

    + [Contexte et objectifs](#Contexte-et-objectifs)
    
    + [Requirements](#Requirements)

* [Datasets](#Datasets)

    + [Description des datasets utilisées](#Composition-des-datasets)

## Introduction

### Contexte et objectifs

Pour notre projet de Data Mining, nous avons choisi d'explorer les données issues de la palteforme Open Data du Syndicat des Transports d'Ile-de-France (STIF) disponible à [cette adresse](https://opendata.stif.info/). Nous avons décidé d'explorer les données issues de la validation des voyageurs sur les différents réseaux ferrés d'Ile-de-France.

Notre objectif est de mener une double étude sur les caractéristiques actuelles du réseau ferré et de son efficacité. Nous allons donc d'une part appliquer des algorithmes de clustering sur les différentes stations du réseau afin de mettre en évidence les profils types. Nous allons d'autre part essayer de recréer des lignes de transports reliant les différentes stations et comparer les résultats obtenus au réseau actuellement en place.

### Requirements

Pour que notre projet fonctionne correctement, nous avons besoin d'importer les libraries suivantes :

In [1]:
import csv
import folium
import pandas as pd
import numpy as np

## Datasets

### Description des datasets

Parmis les jeux de données mis à disposition par la plateforme Open Data du STIF, nous avons choisi d'utiliser les 4 types de datasets suivants :

- [validations-nombre-par-jour-2017s1](https://opendata.stif.info/explore/dataset/validations-sur-le-reseau-ferre-nombre-de-validations-par-jour-1er-sem/) et [validations-nombre-par-jour-2017s2](https://opendata.stif.info/explore/dataset/validations-sur-le-reseau-ferre-profils-horaires-par-jour-type-2e-sem/) : contiennent le nombre de validations par jour par station par catégorie de titre de transport pour chaqeu semestre de 2017.

- [validations-profils-horaires-2017s1](https://opendata.stif.info/explore/dataset/validations-sur-le-reseau-ferre-profils-horaires-par-jour-type-1er-sem/) et [validations-profils-horaires-2017s2](https://opendata.stif.info/explore/dataset/validations-sur-le-reseau-ferre-profils-horaires-par-jour-type-2e-sem/) : contiennent les pourcentages moyens de validation par tranche horaire à la station donnée suivant le type de journée (jour ordinaire, dimanche, etc...)  pour chaque semestre de 2017.

- [emplacement-des-gares-idf](https://opendata.stif.info/explore/dataset/emplacement-des-gares-idf/) : contient les emplacements de l'ensemble des stations du réseau ferré d'Ile-de-France.

- [referentiel-arret-tc-idf-](https://opendata.stif.info/explore/dataset/referentiel-arret-tc-idf/) : recense l'ensemble des stations des réseaux de surface et ferré de transport d'Ile-de-France suivant différents niveaux de granularités.

Tous ces datasets sont disponibles dans le dossier `Data` sous forme de fichiers csv.

### Composition des datasets

Nous allons décrire dans cette partie les différents champs que nous allons utiliser pour chaque type de 

#### **Datasets du nombre de validations par jour**

Les champs que nous avons choisi de garder sont représentés dans le tableau suivant :

| Nom du champs | Description | Valeur |
|---------------|-------------|--------|
| LIBELLE_ARRET | Libellée de l'arrêt | string |
| ID_REFA_LDA | Id de l'arrêt | float |
| JOUR | Jour de l'année | string |
| CATEGORIE_TITRE | Catégorie du titre | AMETHYSTE, AUTRE TITRE, FGT, IMAGINE R, NAVIGO, TST |
| NB_VALD | Nombre de validations | int ou 'Moins de 5' |

#### **Datasets du profil de validation**

| Nom du champs | Description | Valeur |
|---------------|-------------|--------|
| LIBELLE_ARRET | Libellée de l'arrêt | string |
| ID_REFA_LDA | Id de l'arrêt | float |
| CAT_JOUR | Type de journée | JOHV, SAHV, JOVS, SAVS, DIJFP |
| TRNC_HORR_60 | Tranche horaire | Interval (ex: 1H - 2H) |
| pourc_validations | Pourcentage de validations | float |